<a href="https://colab.research.google.com/github/OLIX-111/CuentaCuentosIA/blob/main/cuentaCuentos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#CuentaCuentosIA


!pip install openai gradio python-dotenv

# Importaciones
import os
import gradio as gr
from openai import OpenAI

# Configuración de la API de OpenAI
# IMPORTANTE: En Colab, usaremos getpass para ingresar la API Key de forma segura
from google.colab import userdata

class GeneradorCuentos:
    def __init__(self, client):
        self.client = client
        self.modelo = "gpt-3.5-turbo"

    def generar_cuento(self, palabra_clave):
        try:
            respuesta = self.client.chat.completions.create(
                model=self.modelo,
                messages=[
                    {"role": "system", "content": "Eres un escritor creativo que genera cuentos cortos con una estructura narrativa clara: introducción, nudo y desenlace."},
                    {"role": "user", "content": f"Genera un cuento corto de 3 párrafos inspirado en: '{palabra_clave}'. El cuento debe tener una introducción que presente el escenario, un nudo que desarrolle un conflicto, y un desenlace que resuelva la historia."}
                ],
                max_tokens=500,
                temperature=0.7
            )

            cuento = respuesta.choices[0].message.content
            return cuento

        except Exception as e:
            return f"Error al generar el cuento: {str(e)}"

class GeneradorImagenes:
    def __init__(self, client):
        self.client = client
        self.modelo = "dall-e-3"

    def generar_imagen_portada(self, cuento):
        try:
            # Generar un prompt descriptivo para la imagen
            respuesta_prompt = self.client.chat.completions.create(
                model="gpt-3.5-turbo",
                messages=[
                    {"role": "system", "content": "Eres un generador de prompts creativos para imágenes."},
                    {"role": "user", "content": f"Genera un prompt descriptivo y visual para una imagen de portada basada en este cuento: '{cuento}'. El prompt debe ser conciso y capturar la esencia de la historia."}
                ]
            )

            prompt_imagen = respuesta_prompt.choices[0].message.content

            # Generar imagen con DALL-E
            respuesta_imagen = self.client.images.generate(
                model=self.modelo,
                prompt=prompt_imagen,
                size="1024x1024",
                quality="standard",
                n=1
            )

            url_imagen = respuesta_imagen.data[0].url
            return url_imagen

        except Exception as e:
            return f"Error al generar la imagen: {str(e)}"

# Función para configurar el cliente de OpenAI
def configurar_cliente_openai():
    try:
        # Intentar obtener la clave API de los secretos de Colab
        api_key = userdata.get('OPENAI_API_KEY')
    except:
        # Si no está configurado en secretos, solicitarlo al usuario
        from getpass import getpass
        api_key = getpass("Ingresa tu API Key de OpenAI: ")

    return OpenAI(api_key=api_key)

# Configurar cliente de OpenAI
cliente_openai = configurar_cliente_openai()

# Inicializar generadores
generador_cuentos = GeneradorCuentos(cliente_openai)
generador_imagenes = GeneradorImagenes(cliente_openai)

def generar_cuento_con_imagen(palabra_clave):
    try:
        # Generar cuento
        cuento = generador_cuentos.generar_cuento(palabra_clave)

        # Generar imagen de portada
        url_imagen = generador_imagenes.generar_imagen_portada(cuento)

        return cuento, url_imagen

    except Exception as e:
        return f"Error: {str(e)}", None

# Crear interfaz Gradio
def crear_interfaz():
    with gr.Blocks() as demo:
        gr.Markdown("# 📖 Generador de Cuentos con IA")

        with gr.Row():
            input_palabra = gr.Textbox(label="Palabra o Título del Cuento")
            btn_generar = gr.Button("Generar Cuento")

        output_cuento = gr.Textbox(label="Cuento Generado", lines=10)
        output_imagen = gr.Image(label="Imagen de Portada")

        btn_generar.click(
            fn=generar_cuento_con_imagen,
            inputs=[input_palabra],
            outputs=[output_cuento, output_imagen]
        )

    return demo

# Lanzar la aplicación
interfaz = crear_interfaz()
interfaz.launch(share=True)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.1/57.1 MB 19.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.1/320.1 kB 28.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.1/11.1 MB 112.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 12.1 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2
Ingresa tu API Key de OpenAI: ··········
Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://3fbef5181826330ad8.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run